# Save Images

In [ ]:
from google.colab import drive
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
import random

In [ ]:
drive.mount('/content/drive')

In [ ]:
covidPath = "/content/drive/MyDrive/project-yr-3/covid/"
tbPath = "/content/drive/MyDrive/project-yr-3/tb/"
normalPath = "/content/drive/MyDrive/project-yr-3/normal/"

random.shuffle(os.listdir(covidPath))
random.shuffle(os.listdir(tbPath))
random.shuffle(os.listdir(normalPath))

In [ ]:
imageList = []
img_size = 224
total_imgs = 1000

# extract files from covid data folders
def extract_images(path):
  count = 0
  for img in os.listdir(path):
    if (count < (total_imgs/4)/3):
      img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
      new_array = cv2.resize(img_array, (img_size, img_size))
      #new_array = cv2.GaussianBlur(new_array, (random.randrange(1, 25, 2), random.randrange(1, 25, 2)), 0)
      imageList.append(new_array.copy())
      count += 1

extract_images(covidPath)
extract_images(tbPath)
extract_images(normalPath)

In [ ]:
# show first image in image list
print("Total images: " + str(len(imageList)) + "\n")
print("Image 1:")
print(imageList[0].shape)
plt.imshow(imageList[0], cmap='gray')
plt.show()

In [ ]:
grid_size = 2
block_size = img_size//grid_size

#reshape into 4 blocks
def blockify(image):
  square_blocks = image.reshape(grid_size, block_size, grid_size, block_size,).swapaxes(2, 1)
  square_blocks = square_blocks.reshape(-1, block_size, block_size)
  return square_blocks

In [ ]:
img1 = blockify(imageList[0])
print(np.shape(img1))

for i, block in enumerate(img1):
    plt.figure(10+i//9, figsize = (10, 10))
    plt.subplot(grid_size, grid_size, i%9+1)
    plt.imshow(block, cmap='gray')
    plt.title("Block " + str(i+1))

In [ ]:
# create directories for each label
mainDir = '/content/gdrive/imgs/'

for i in range(1,5):
  if not os.path.exists(mainDir + str(i) + '/'):
      os.makedirs(mainDir + str(i) + '/')

In [ ]:
# add list images to corresponding directories
myPath = os.path.abspath(mainDir)

j = 1
ind = 0
while ind < len(imageList):
  blocks = blockify(imageList[ind])
  for i, block in enumerate(blocks):
    plt.imsave(myPath + '/' + str(i+1) + '/' + str(j) + '.jpg', block)
    j += 1
  ind += 1

In [ ]:
#Counting images in directories
total = 0
print("Number of images per class: ")
for i in range(1,5):
  num = len(os.listdir(mainDir + '/' + str(i)))
  total = total + num
  print("Block " + str(i) + ": " + str(num))
print("Total images: "+ str(total) + '\n')

# Load Images

In [ ]:
import random

In [ ]:
training_data = []

def create_training_data():
  for i in range(0,4):
    path = os.path.join(mainDir, str(i+1))
    class_num = i
    for img in os.listdir(path):
      try:
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        new_array = cv2.resize(img_array, (img_size, img_size))
        #new_array = cv2.GaussianBlur(new_array, (random.randrange(1, 25, 2), random.randrange(1, 25, 2)), 0)
        training_data.append([new_array, class_num])
      except Exception as e:
        pass

create_training_data()

In [ ]:
print(len(training_data))

In [ ]:
random.shuffle(training_data)

In [ ]:
for sample in training_data[:10]:
  print(sample[1])

In [ ]:
X = []
y = []

In [ ]:
for features, label in training_data:
  X.append(features)
  y.append(label)

X = np.array(X).reshape(-1, img_size, img_size, 1)
y = np.array(y)

# Training

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import regularizers

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7)

In [ ]:
#prepare data for training
#normalise value
X_train = X_train/255.0
X_test = X_test/255.0

#transform labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 4)
y_test = keras.utils.to_categorical(y_test, 4)

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=X_train.shape[1:]))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))

model.add(keras.layers.Dense(4, activation='softmax'))

model.summary()
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics='acc')

# Select and Fit Model

In [ ]:
#callback for 100% accuracy
import tensorflow as tf
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_acc')==1.0):
      print("\nReached 100% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
#dot_img_file = '/tmp/model_1.png'
#tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1, callbacks=[callbacks])

model.save_weights('/content/drive/MyDrive/project-yr-3/grid_location_model_weights.h5')

# Results

In [ ]:
from sklearn import metrics
import seaborn as sn
import pandas as pd

In [ ]:
f, ax = plt.subplots()
ax.plot([None] + history.history['acc'], 'o-')
ax.plot([None] + history.history['val_acc'], 'x-')

#plot legend and use the best location automatically: loc = 0
ax.legend(['Training Accuracy', 'Validation Accuracy'], loc = 0)
ax.set_title('Training/Validation Accuracy Per Epoch \nPretext Task 2: Grid Location Classification')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')

In [ ]:
f, ax = plt.subplots()
ax.plot([None] + history.history['loss'], 'o-')
ax.plot([None] + history.history['val_loss'], 'x-')

#plot legend and use the best location automatically: loc = 0
ax.legend(['Training Loss', 'Validation Loss'], loc = 0)
ax.set_title('Training/Validation Loss Per Epoch \nPretext Task 2: Grid Location Classification')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')

In [ ]:
score = model.evaluate(X_test, y_test, verbose = 0) 

print(model.metrics_names)
print(score)

print('Test loss:', round(score[0], 4)) 
print('Test accuracy:', round(score[1], 4))

In [ ]:
y_pred = model.predict(X_test)

labels = ['block 1', 'block 2', 'block 3', 'block 4']
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(matrix)

matrix_norm = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1), normalize='true')
print(matrix_norm)

In [ ]:
df_cfm = pd.DataFrame(matrix, index = labels, columns = labels)
plt.figure(figsize = (10,7))
cfm_plot = sn.heatmap(df_cfm, annot=True, fmt='g')
cfm_plot.set_title('Confusion Matrix \nPretext Task 2: Grid Location Classification')
cfm_plot.set_xlabel('Predicted Label')
cfm_plot.set_ylabel('True Label')

df_cfm = pd.DataFrame(matrix_norm, index = labels, columns = labels)
plt.figure(figsize = (10,7))
cfm_plot = sn.heatmap(df_cfm, annot=True, fmt='g')
cfm_plot.set_title('Normalised Confusion Matrix \nPretext Task 2: Grid Location Classification')
cfm_plot.set_xlabel('Predicted Label')
cfm_plot.set_ylabel('True Label')

In [ ]:
#test out-of-sample images by SEPERATE UPLOAD
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

def find_class(classes):
  maxValue = np.amax(classes)
  index = np.where(classes == np.amax(classes))
  result = ''
  if index[1] == 0:
    result = "1"
  elif index[1] == 1:
    result = "2"
  elif index[1] == 2:
    result = "3"
  else:
    result = "4"
  return result

for fn in uploaded.keys():
  path = fn
  img_array = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  new_array = cv2.resize(img_array, (img_size, img_size))
  x = image.img_to_array(new_array)
  x = np.expand_dims(x, axis = 0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=1)
  print(fn)

  plt.imshow(new_array, cmap='gray')
  plt.title("prediction: Block " + str(find_class(classes)))
  plt.show()